In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

/home/adminroot/miniconda3/envs/distill_data/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
training_data=datasets.FashionMNIST(
    root="fun_data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data=datasets.FashionMNIST(
    root="fun_data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [3]:
train_dataloader=DataLoader(training_data,batch_size=64,shuffle=True)
test_dataloader=DataLoader(test_data,batch_size=64)

In [4]:
tmp=next(iter(train_dataloader))
tmp[0].size()

torch.Size([64, 1, 28, 28])

In [5]:
print(len(train_dataloader))
print(len(training_data))

938
60000


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model=NeuralNetwork()

In [7]:
loss_fn=nn.CrossEntropyLoss()

In [8]:
# optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
optimizer=torch.optim.SGD(model.parameters(),lr=0.01,momentum=0.9)
scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.9)
optimizer,scheduler

(SGD (
 Parameter Group 0
     dampening: 0
     differentiable: False
     foreach: None
     initial_lr: 0.01
     lr: 0.01
     maximize: False
     momentum: 0.9
     nesterov: False
     weight_decay: 0
 ),
 <torch.optim.lr_scheduler.ExponentialLR at 0x7fb35bd7e550>)

In [9]:
def train(dataloader,model,loss_fn,optimizer):
    size=len(dataloader.dataset)
    model.train()
    #model train is set for batch normalization
    #and dropout layers
    for batch,(x,y) in enumerate(dataloader):
        optimizer.zero_grad()
        pred=model(x)
        loss=loss_fn(pred,y)
        loss.backward()
        optimizer.step()
        if batch%100==0:
            loss,current=loss.item(),(batch+1)*len(x)
            print(f"loss: {loss} [{current}/{size}]")
    

In [10]:
def test(dataloader,model,loss_fn):
    model.eval()
    size=len(dataloader.dataset) #all data
    num_batches=len(dataloader) # num of batch
    test_loss,correct=0,0
    with torch.no_grad():
        for x,y in dataloader:
            pred=model(x)
            test_loss+=loss_fn(pred,y).item()
            correct+=(pred.argmax(1)==y).sum().item()
    test_loss/=num_batches
    correct/=size
    print(f"test error: \n Accuracy: {100*correct}, Avg loss: {test_loss}")

In [11]:
epochs=10
for t in range(epochs):
    print(f"Epochs {t+1}\n----------------------")
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader,model,loss_fn)
    scheduler.step()
print("Done!")

Epochs 1
----------------------
loss: 2.299267053604126 [64/60000]
loss: 0.7538191676139832 [6464/60000]
loss: 0.6451717019081116 [12864/60000]
loss: 0.5404155850410461 [19264/60000]
loss: 0.5290641784667969 [25664/60000]
loss: 0.5158385038375854 [32064/60000]
loss: 0.4327242970466614 [38464/60000]
loss: 0.542901337146759 [44864/60000]
loss: 0.39552080631256104 [51264/60000]
loss: 0.6330477595329285 [57664/60000]
test error: 
 Accuracy: 81.11, Avg loss: 0.5267184053067189
Epochs 2
----------------------
loss: 0.29002639651298523 [64/60000]
loss: 0.47462591528892517 [6464/60000]
loss: 0.6206664443016052 [12864/60000]
loss: 0.43456897139549255 [19264/60000]
loss: 0.33329957723617554 [25664/60000]
loss: 0.42221033573150635 [32064/60000]
loss: 0.3988877236843109 [38464/60000]
loss: 0.31969013810157776 [44864/60000]
loss: 0.39035317301750183 [51264/60000]
loss: 0.33082470297813416 [57664/60000]
test error: 
 Accuracy: 84.82, Avg loss: 0.4130552054210833
Epochs 3
----------------------
loss:

In [12]:
torch.save(model.state_dict(),'checkpoint.pth')

In [15]:
import time
start_time=time.time()
state_dict=torch.load('checkpoint.pth')
end_time=time.time()
print(f"loading time without mmap={end_time-start_time}")

loading time without mmap=0.00666499137878418


In [16]:
start_time=time.time()
state_dict=torch.load('checkpoint.pth',mmap=True)
end_time=time.time()
print(f"loading time with the mmap={end_time-start_time}")

loading time with the mmap=0.004232883453369141
